In [43]:
import numpy as np
import pandas as pd
import statsmodels.tsa.seasonal as STL
import matplotlib.pyplot as plt

testData_01 = pd.read_csv("./TestdatenSet1.csv")
print(testData_01)

# Do one with electrity and one with voltage



       Id              Zeitstempel MaschinenID           MachineLabel  \
0      96  2025-08-04 10:49:15.287  HELLER_PT5  IGH_ProfiTrainer_PT16   
1      97  2025-08-04 11:22:01.020  HELLER_PT5  IGH_ProfiTrainer_PT16   
2      98  2025-08-04 11:24:27.807  HELLER_PT5  IGH_ProfiTrainer_PT16   
3      99  2025-08-04 11:26:07.867  HELLER_PT5  IGH_ProfiTrainer_PT16   
4     100  2025-08-04 11:30:28.233  HELLER_PT5  IGH_ProfiTrainer_PT16   
..    ...                      ...         ...                    ...   
995  1091  2025-08-21 12:19:32.197  HELLER_PT5  IGH_ProfiTrainer_PT16   
996  1092  2025-08-21 12:19:39.117  HELLER_PT5  IGH_ProfiTrainer_PT16   
997  1093  2025-08-21 12:19:44.327  HELLER_PT5  IGH_ProfiTrainer_PT16   
998  1094  2025-08-21 12:19:49.933  HELLER_PT5  IGH_ProfiTrainer_PT16   
999  1095  2025-08-21 12:23:37.007  HELLER_PT5  IGH_ProfiTrainer_PT16   

        OrderID  ConfirmationID MaterialID Tools  Component    ProgramName  \
0    60006833.0        220621.0     SP3789   

In [ ]:
# Strom pipeline
testData_01_power_df = testData_01[["Zeitstempel", "Strom"]].copy()
# We will assume for now that NaN is equivalent to the machine just not running; optional: we could try to build a missing value handler 
# based on linear interpolation (does that even make sense with electricity?) or maybe by estimating a value via loess regression
# set up dataframe with timestamp as an index 
testData_01_power_df.rename(columns= {"Zeitstempel": "Timestamp", "Strom": "Power"}, inplace = True)
testData_01_power_df["Power"] = testData_01_power_df["Power"].str.replace(",", ".", regex=False)
testData_01_power_df["Power"] = pd.to_numeric(testData_01_power_df["Power"], errors = "raise") # if errors then its put into an NaN state
testData_01_power_df.dropna(subset=["Power"], inplace = True)
# print(testData_01_power_df)

testData_01_power_df["Timestamp"] = pd.to_datetime(testData_01_power_df["Timestamp"])
testData_01_power_df.set_index("Timestamp", inplace=True)
# print(testData_01_power_df)
print(testData_01_power_df["Power"].dtype)
print(testData_01_power_df.index.dtype)

# first step stl
stl_testData_power = STL(testData_01_power_df["Power"], period = 60)
res = stl_testData_power.fit()

res.plot()
plt.show()

float64
datetime64[ns]
Timestamp
0 days 00:00:06.020000      39
0 days 00:00:06.010000      29
0 days 00:00:06.017000      29
0 days 00:00:06.023000      21
0 days 00:00:06.013000      19
                            ..
0 days 00:00:09.004000       1
0 days 00:00:06.106000       1
0 days 00:00:06.104000       1
-1 days +23:44:55.120000     1
7 days 22:40:14.157000       1
Name: count, Length: 87, dtype: int64


TypeError: 'module' object is not callable